# Kepler Framework, Emulate v16

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'


In [4]:
tpath_data17 = 'output/emulation/data17_table_v16.h5'
tpath_data18 = 'output/emulation/data18_table_v16.h5'


## Setup Chains:

In [5]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v16_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v16_ivarloose" , L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v16_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v16_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v16"          , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v16_loose' ),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v16_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v16_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v16_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v16_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [6]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [7]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:27<00:00, 21.96s/it]


In [8]:
data17_df.shape

(43311283, 112)

### Validate:

In [9]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v16.root')

2022-03-24 11:10:17,604 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v16.root


In [10]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v16'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:53<00:00, 22.64s/it]


HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5333  | 29975133/32047563 |
|   L2   | 93.2899  | 29897131/32047563 |
| EFCalo | 91.9789  | 29477011/32047563 |
|  HLT   | 90.0347  | 28853935/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:26<00:00, 17.21s/it]


HLT_e28_lhtight_nod0_ringer_v16_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2316  | 21452391/22765602 |
|   L2   | 94.1153  | 21425925/22765602 |
| EFCalo | 93.1905  | 21215376/22765602 |
|  HLT   | 87.0016  | 19806445/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 96.8625  | 1324872/1367786 |
|   L2   | 96.7714  | 1323625/1367786 |
| EFCalo | 94.6781  | 1294994/1367786 |
|  HLT   | 91.4098  | 1250291/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.41it/s]

HLT_e140_lhloose_nod0_ringer_v16
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 96.1282  | 58197/60541  |
|   L2   | 96.0638  | 58158/60541  |
| EFCalo | 94.7754  | 57378/60541  |
|  HLT   | 93.0312  | 56322/60541  |
+--------+----------+--------------+


In [11]:
eff_data17_Zee.save()

In [12]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v16.root')

2022-03-24 11:14:42,028 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v16.root


In [13]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v16'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.34s/it]


HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  0.9638  |  99751/10349992  |
|   L2   |  0.9209  |  95308/10349992  |
| EFCalo |  0.8158  |  84439/10349992  |
|  HLT   |  0.0815  |  8434/10349992   |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.90s/it]


HLT_e28_lhtight_nod0_ringer_v16_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.3100  |  28689/9254300  |
|   L2   |  0.2859  |  26456/9254300  |
| EFCalo |  0.2380  |  22027/9254300  |
|  HLT   |  0.0009  |    79/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.30s/it]


HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.1947  |  15287/7852181  |
|   L2   |  0.1656  |  13005/7852181  |
| EFCalo |  0.1079  |   8476/7852181  |
|  HLT   |  0.0006  |    48/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.45s/it]

HLT_e140_lhloose_nod0_ringer_v16
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  0.4704  |  26502/5634041  |
|   L2   |  0.4156  |  23416/5634041  |
| EFCalo |  0.2887  |  16267/5634041  |
|  HLT   |  0.0134  |   754/5634041   |
+--------+----------+-----------------+


In [14]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [16]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [17]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:23<00:00, 20.82s/it]


In [18]:
data18_df.shape

(41873973, 112)

In [19]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v16.root')

2022-03-24 11:18:45,813 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v16.root


In [20]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v16'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:56<00:00, 23.25s/it]


HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5781  | 30525649/32620495 |
|   L2   | 93.3400  | 30447956/32620495 |
| EFCalo | 92.0065  | 30012984/32620495 |
|  HLT   | 90.5615  | 29541622/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:25<00:00, 17.08s/it]


HLT_e28_lhtight_nod0_ringer_v16_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2037  | 21962497/23313826 |
|   L2   | 94.1073  | 21940010/23313826 |
| EFCalo | 93.3073  | 21753509/23313826 |
|  HLT   | 87.5797  | 20418176/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 96.9011  | 1338858/1381675 |
|   L2   | 96.8109  | 1337612/1381675 |
| EFCalo | 94.7192  | 1308711/1381675 |
|  HLT   | 91.7457  | 1267628/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.49it/s]

HLT_e140_lhloose_nod0_ringer_v16
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 96.2013  | 58095/60389  |
|   L2   | 96.1168  | 58044/60389  |
| EFCalo | 94.9693  | 57351/60389  |
|  HLT   | 93.3647  | 56382/60389  |
+--------+----------+--------------+


In [21]:
eff_data18_Zee.save()

In [22]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v16.root')

2022-03-24 11:23:10,509 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v16.root


In [23]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v16'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.50s/it]


HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  1.2817  |  108520/8467098 |
|   L2   |  1.2405  |  105033/8467098 |
| EFCalo |  1.1365  |  96232/8467098  |
|  HLT   |  0.4194  |  35508/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.16s/it]


HLT_e28_lhtight_nod0_ringer_v16_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.5313  |  40533/7628804  |
|   L2   |  0.5084  |  38784/7628804  |
| EFCalo |  0.4615  |  35210/7628804  |
|  HLT   |  0.0277  |   2113/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.71s/it]


HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.3784  |  24833/6561985  |
|   L2   |  0.3508  |  23021/6561985  |
| EFCalo |  0.2956  |  19395/6561985  |
|  HLT   |  0.0302  |   1983/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.06s/it]

HLT_e140_lhloose_nod0_ringer_v16
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  0.7786  |  37191/4776518  |
|   L2   |  0.7248  |  34621/4776518  |
| EFCalo |  0.6023  |  28769/4776518  |
|  HLT   |  0.2155  |  10292/4776518  |
+--------+----------+-----------------+


In [24]:
eff_data18_JF17.save()